<a href="https://colab.research.google.com/github/TimotheeeNiven/IntroML_TNiven/blob/main/Homework6_TNiven.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import time
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import pandas as pd
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Problem 1
urlhousing = '/content/drive/MyDrive/IntroML/CSVFiles/Housing.csv'

df = pd.read_csv(urlhousing)

# Display the first 5 rows of the DataFrame
print(df.head())

      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  


In [ ]:
# List of variables to map

varlist =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

# Defining the map function
def binary_map(x):
    return x.map({'yes': 1, 'no': 0, 'furnished': 2, 'semi-furnished': 1, 'unfurnished': 0})
# Defining the map function for furnishing status

# Applying the function to the housing list
df[varlist] = df[varlist].apply(binary_map)
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2


In [ ]:
from sklearn.preprocessing import StandardScaler
import statistics
# Calculate the standard deviation
columns = ['price','area','bedrooms','bathrooms','stories','mainroad','guestroom','basement','hotwaterheating','airconditioning','parking','prefarea','furnishingstatus']

scaler = StandardScaler()

# Fit and transform the scaler into the forementioned calculation
df[columns] = scaler.fit_transform(df[columns])
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,4.566365,1.046726,1.403419,1.421812,1.378217,0.405623,-0.465315,-0.734539,-0.219265,1.472618,1.517692,1.804941,1.406286
1,4.004484,1.757010,1.403419,5.405809,2.532024,0.405623,-0.465315,-0.734539,-0.219265,1.472618,2.679409,-0.554035,1.406286
2,4.004484,2.218232,0.047278,1.421812,0.224410,0.405623,-0.465315,1.361397,-0.219265,-0.679063,1.517692,1.804941,0.091662
3,3.985755,1.083624,1.403419,1.421812,0.224410,0.405623,-0.465315,1.361397,-0.219265,1.472618,2.679409,1.804941,1.406286
4,3.554979,1.046726,1.403419,-0.570187,0.224410,0.405623,2.149083,1.361397,-0.219265,1.472618,1.517692,-0.554035,1.406286


In [ ]:
#Data set locations
X = df.iloc[:, [1,2,3,4,5,6,7,8,9,10,11]].values
Y = df.iloc[:, 0].values
#Split the data set
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.float32).view(-1, 1)
X_train, X_val, y_train, y_val = train_test_split(X_tensor, Y_tensor, test_size=0.2, random_state=42)

In [ ]:
#Very Fun Neural Network Model
linear_model = nn.Linear(1,1)
optimizer = optim.SGD(linear_model.parameters(),lr=1e-2)

#This is our Training Loop
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,t_c_train,t_c_val):
  for epoch in range(1, n_epochs + 1):
    t_p_train = model(t_u_train)
    loss_train = loss_fn(t_p_train, t_c_train)
    t_p_val = model(t_u_val)
    loss_val = loss_fn(t_p_val,t_c_val)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch == 1 or epoch % 1000 == 0:
      print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"f" Validation loss {loss_val.item():.4f}")

In [ ]:
#Hidden Layer Portion
seq_model = nn.Sequential(nn.Linear(11,13),nn.Tanh(),nn.Linear(13,11))

y_train = y_train.expand(-1, 11)
y_val = y_val.expand(-1, 11)
#Running Our Training Loop
optimizer = optim.SGD(seq_model.parameters(),lr=1e-2)
training_loop(n_epochs = 3000, optimizer = optimizer, model = seq_model, loss_fn = nn.MSELoss(), t_u_train = X_train, t_u_val = X_val, t_c_train = y_train, t_c_val = y_val)
print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 1.0419, Validation loss 1.6577
Epoch 1000, Training loss 0.3246, Validation loss 0.5748
Epoch 2000, Training loss 0.3034, Validation loss 0.5440
Epoch 3000, Training loss 0.2933, Validation loss 0.5322

Parameter containing:
tensor([[0.7288]], requires_grad=True)
Parameter containing:
tensor([-0.3327], requires_grad=True)


In [ ]:
#Sequential Model Part B
seq_model2 = nn.Sequential(
    nn.Linear(11, 32),
    nn.Tanh(),
    nn.Linear(32, 64),
    nn.Tanh(),
    nn.Linear(64, 16),
    nn.Tanh(),
    nn.Linear(16, 11)
)

#Running Our Training Loop
optimizer = optim.SGD(seq_model2.parameters(),lr=1e-2)
training_loop(n_epochs = 3000, optimizer = optimizer, model = seq_model2, loss_fn = nn.MSELoss(), t_u_train = X_train, t_u_val = X_val, t_c_train = y_train, t_c_val = y_val)
print()
print(linear_model.weight)
print(linear_model.bias)

Epoch 1, Training loss 0.9026, Validation loss 1.4605
Epoch 1000, Training loss 0.3243, Validation loss 0.5871
Epoch 2000, Training loss 0.3015, Validation loss 0.5642
Epoch 3000, Training loss 0.2832, Validation loss 0.5475

Parameter containing:
tensor([[0.7288]], requires_grad=True)
Parameter containing:
tensor([-0.3327], requires_grad=True)


#Problem 2

In [ ]:
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# Define the neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Instantiate the model and move it to the device
model = SimpleNN().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
# Training loop
start_time = time.time()
num_epochs = 10

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(trainloader)}")

end_time = time.time()
training_time = end_time - start_time

Epoch 1/10, Loss: 1.8737832964838619
Epoch 2/10, Loss: 1.6543723300594808
Epoch 3/10, Loss: 1.569978554840283
Epoch 4/10, Loss: 1.5126125516793918
Epoch 5/10, Loss: 1.4665346935277095
Epoch 6/10, Loss: 1.4261280492596005
Epoch 7/10, Loss: 1.3894387744271848
Epoch 8/10, Loss: 1.3566789803907389
Epoch 9/10, Loss: 1.3254558856377516
Epoch 10/10, Loss: 1.2968280111889705


In [ ]:
# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total

print(f"Training Time: {training_time:.2f} seconds")
print(f"Training Loss: {running_loss / len(trainloader)}")
print(f"Evaluation Accuracy: {accuracy * 100:.2f}%")

Training Time: 268.88 seconds
Training Loss: 1.2968280111889705
Evaluation Accuracy: 51.95%


In [ ]:
#Problem 2B
# Define the neural network
class SimpleNN2(nn.Module):
    def __init__(self):
        super(SimpleNN2, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(512, 1024)
        self.fc3 = nn.Linear(1024,256)
        self.fc4 = nn.Linear(256,10)
        # Similar to problem 1 we implimented multiple layers in this section of the code

    def forward(self, x):
            x = x.view(-1, 32 * 32 * 3)
            x = self.relu(self.fc1(x))
            x = self.relu(self.fc2(x))
            x = self.relu(self.fc3(x))
            x = self.fc4(x)
            return x

# Instantiate the model and move it to the device
model2 = SimpleNN2().to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model2.parameters(), lr=0.01)

In [ ]:
# Training loop
start_time = time.time()
num_epochs = 300

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(trainloader)}")

end_time = time.time()
training_time = end_time - start_time

Epoch 1/300, Loss: 2.171310120988685
Epoch 2/300, Loss: 1.870802794271113
Epoch 3/300, Loss: 1.7156176632627502
Epoch 4/300, Loss: 1.6198466090899903
Epoch 5/300, Loss: 1.54624485923811
Epoch 6/300, Loss: 1.478658675842578
Epoch 7/300, Loss: 1.4196027252070433
Epoch 8/300, Loss: 1.36771037069428
Epoch 9/300, Loss: 1.3207042850649264
Epoch 10/300, Loss: 1.275192771421369
Epoch 11/300, Loss: 1.2366359996826142
Epoch 12/300, Loss: 1.1957905706389786
Epoch 13/300, Loss: 1.1595185936411934
Epoch 14/300, Loss: 1.1241462747459217
Epoch 15/300, Loss: 1.0868778189124964
Epoch 16/300, Loss: 1.0521662451727005
Epoch 17/300, Loss: 1.0188714561559964
Epoch 18/300, Loss: 0.9895027704403528
Epoch 19/300, Loss: 0.9549293002813978
Epoch 20/300, Loss: 0.9228993962945231
Epoch 21/300, Loss: 0.8892044592696382
Epoch 22/300, Loss: 0.8606198347742905
Epoch 23/300, Loss: 0.8258611928776401
Epoch 24/300, Loss: 0.7960970186440232
Epoch 25/300, Loss: 0.7650764172186937
Epoch 26/300, Loss: 0.736130756604702
Epoc

In [ ]:
# Evaluation
model2.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model2(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total

print(f"Training Time: {training_time:.2f} seconds")
print(f"Training Loss: {running_loss / len(trainloader)}")
print(f"Evaluation Accuracy: {accuracy * 100:.2f}%")

Training Time: 9828.96 seconds
Training Loss: 0.00023028890038748174
Evaluation Accuracy: 55.12%
